In [2]:
#%pip install pandas requests ipykernel  

In [6]:
#Examples in cells below of how to compare across Awareness Comp and Halo
#Run first to create helper funcs

import pandas as pd
import qualtricsapi as qa
    
token=qa.get_secrets()


df_aw=qa.getResults(qa.SetSurveyId.survey_choices['Awareness'],token)
df_aw.to_csv('aw.csv')

df_comp = qa.getResults(qa.SetSurveyId.survey_choices['Competency'],token)
df_comp.to_csv('comp.csv')



def eligible_comp_track(track, level = "Level One", OIT_only = None , Halo = True, df_aw=None, df_comp=None):
    if df_aw is None or df_aw.empty:
        df_aw=qa.getResults(qa.SetSurveyId.survey_choices['Awareness'],token)
        #df_aw.to_csv('aw.csv')
    if df_comp is None or df_comp.empty:
        df_comp = qa.getResults(qa.SetSurveyId.survey_choices['Competency'],token)
        #df_comp.to_csv('comp.csv')

    aw_query = "Track == @track & Dropped != 'Yes'"
    comp_query = "Track == @track & Dropped != 'Yes' & Level == @level"
    #To do: Add support for multiple Levels

    if Halo:
        aw_query = aw_query + "& Halo == 'Yes'"

    if OIT_only == 'Only':
        aw_query = aw_query + "& Group != 'Other (please specify)'"
        comp_query = comp_query + "& Group != 'Other (please specify)'"
    
    elif OIT_only == 'Other':
        aw_query = aw_query + "& Group == 'Other (please specify)'"
        comp_query = comp_query + "& Group == 'Other (please specify)'"

    else:
        print(f"OIT only {OIT_only} not a valid option, ASSUMING ALL")

   

    df_eligible = (df_aw.query(aw_query)
                .merge(df_comp.query(comp_query),how='left', left_on='Email Embedded', right_on='Email Embedded', indicator=True)
                .query("_merge == 'left_only'")
                .loc[:,'Email Embedded']
                .to_frame()
                .rename({'Email Embedded': 'Email'}, axis=1)
                #To do: Are we copying the dataframe?
                .reset_index(drop = True)
    )
    return df_eligible

def eligible_comp_all(df_aw=None, df_comp=None, token=None, OIT_only = None, Halo = True):
    if df_aw is None or df_aw.empty:
        df_aw = qa.getResults(qa.SetSurveyId.survey_choices['Awareness'], token)
        #df_aw.to_csv('aw.csv')
    if df_comp is None or df_comp.empty:
        df_comp = qa.getResults(qa.SetSurveyId.survey_choices['Competency'], token)
        #df_comp.to_csv('comp.csv')
    
    tracks = ['Data Science',
              'Product Management', 'Cloud', 'Human-Centered Design',
              'Cyber-Hygiene: Advanced Topics', 'Leadership',
              'Artificial Intelligence and Machine Learning']
    df_eligible = pd.DataFrame(columns=['track','Email','Group','Halo'])
    for track in tracks:
        df_track = eligible_comp_track(track,df_aw=df_aw,df_comp=df_comp, OIT_only = OIT_only, Halo = Halo)  # make sure eligible_comp_track is defined elsewhere
        df_track['track'] = track
        df_eligible = pd.concat([df_eligible,df_track],ignore_index=True)
    
    return df_eligible


{"result":{"id":"ES_qvsc3doo7518r438o0p017hag2"},"meta":{"httpStatus":"200 - OK","requestId":"0f2d3e66-af42-4310-826b-81a5632c1a8d"}}
Download is 100.0 complete
{"result":{"id":"ES_7v0mr4a3ujg28tpb5qq1a22ep3"},"meta":{"httpStatus":"200 - OK","requestId":"d6f9a5c4-98e2-491f-854b-955d9108daf8"}}
Download is 100.0 complete


In [6]:
'''OIT_only = None is All | OIT_only = Only | OIT_only = Other'''
pd.set_option("display.max_rows", None)
eligible_comp_track('Human-Centered Design', OIT_only = 'None').to_csv('HCD-Comp-VS.csv')

{"result":{"id":"ES_upepolkrabbi8ujn91ortegv08"},"meta":{"httpStatus":"200 - OK","requestId":"ebf0d77d-d65b-46f7-a745-e29ea2784e40"}}
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 100.0 complete
{"result":{"id":"ES_m3qmb6qqpquel9ogglcp8blib0"},"meta":{"httpStatus":"200 - OK","requestId":"1e053685-7cde-4084-a957-f119ee2750f3"}}
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Download is 0.0 complete
Downl

In [7]:
pd.set_option("display.max_rows", None)
comp_output = eligible_comp_all(df_aw=df_aw,df_comp=df_comp,OIT_only = True, Halo = False)
#print(comp_output["Email"].nunique())
comp_output.to_csv("comp_eligible.csv")


OIT only True not a valid option, ASSUMING ALL
OIT only True not a valid option, ASSUMING ALL
OIT only True not a valid option, ASSUMING ALL
OIT only True not a valid option, ASSUMING ALL
OIT only True not a valid option, ASSUMING ALL
OIT only True not a valid option, ASSUMING ALL
OIT only True not a valid option, ASSUMING ALL


In [ ]:
#&  Halo != 'Yes' & `Repeat Learner Embedded` == 'No'"
pd.set_option("display.max_rows", None)
df_no_halo = df_aw.query("`Verified Complete` == 'Yes' & Halo !='Yes' & Group == ['AMG','BOG', 'EADG','ESSG','ICPG','ISPG','IUSG','OIT Front Office']" )
df_no_halo["Email Embedded"].reset_index(drop = True)

In [ ]:
#Find 
df_comp_data = df_comp.query("Track == 'Data Science' & `Verified Complete` == 'Yes'")
df_leadership = (df_aw.query("Track == 'Data Science' & `Verified Complete` == 'Yes' & Halo == 'Yes'")
                .merge(df_comp_leadership,how='left', left_on='Email Embedded', right_on='Email Embedded', indicator=True)
                .query("_merge == 'left_only'")
                .loc[:,'Email Embedded']
)
df_leadership.to_csv("data1.csv")

In [ ]:

df_comp_leadership = df_comp.query("Track == 'Leadership' & `Verified Complete` == 'Yes'")
df_leadership = (df_aw.query("Track == 'Leadership' & Season == 'Spring 2023' & Session == 'Session 2' & Halo == 'Yes'")
               .merge(df_comp_leadership,how='left', left_on='Email Embedded', right_on='Email Embedded', indicator=True)
              .query("_merge == 'left_only'")
              .loc[:,'Email Embedded']
)
df_leadership

In [ ]:
"""
We dont collect this any more
df_pm_complete = (df_aw.query("Track == 'Data Science' & `Verified Complete` == 'Yes' & Halo == 'Yes'"))
df_pm_int = (df_aw.query("`Awareness Interest_3` == 'Somewhat Interested' | `Awareness Interest_3` == 'Yes'")
                .merge(df_pm_complete, how='left', left_on='Email Embedded', right_on='Email Embedded', indicator=True)
                .query("_merge == 'left_only'")
                .loc[:,'Email Embedded']
                .drop_duplicates() )
df_pm_int.to_clipboard()
print(df_pm_int)
"""

In [ ]:
x = None
if x:
    print(x)